In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from IPython.display import HTML
import json
from sklearn.preprocessing import LabelEncoder

import torch.utils.data
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split, GroupKFold, GroupShuffleSplit

In [2]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import LightningModule, LightningDataModule

/opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.9/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: /opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.9/site-packages/torch/lib/libc10.dylib
 in /opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.9/site-packages/torchvision/image.so
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import warnings 
warnings.filterwarnings('ignore')

In [4]:
from datetime import datetime

In [5]:
#skip

In [4]:
data = pd.read_pickle('jd_2_with_macro_v3.pkl')

In [5]:
data_reset = data.reset_index()

In [6]:
target = data_reset[['account','target_24m']]

In [7]:
target_total = target.groupby('account').mean()

In [8]:
features = data_reset[['account','day_payment', 'balance_sum', 'day_debt_bad_max',
                      'ipi1', 'ipi2', 'ppi1', 'ppi2',
       'ppi3', 'cpi1', 'unemp1', 'gdp_nom', 'gdp_yoy', 'salary', 'sales',
       'income_nom', 'income_real_1', 'income_real_2', 'USDRUB1', 'EURRUB1',
       'brent', 'USDRUB_pct', 'brent_pct', 'EURRUB_pct', 'invst', 'key_rate']]

In [9]:
features.account.value_counts().mean()

19.082115754702436

In [10]:
indexes = data.index.get_level_values(0)

In [11]:
x = np.array(indexes)
unique, counts = np.unique(x, return_counts=True)

In [12]:
counts_accs = np.asarray((unique, counts)).T

In [13]:
filter_ = counts_accs[counts_accs[:,1] >= 24][:,0]

In [14]:
filter_df  = data[data.index.get_level_values(0).isin(filter_)]

In [15]:
filter_df.iloc[::2,3:5]

mrf_service_id rf_service_id
account      pay_date                               
2.100100e+11 2017-01-11       12.00000    1204.00000
             2017-03-14       12.00000    1204.00000
             2017-04-17       12.00000    1204.00000
             2017-11-13       12.00000    1204.00000
             2018-01-13       12.00000    1204.00000
...                                ...           ...
8.760108e+11 2020-10-26       11.00000    1116.00000
             2020-12-20       11.00000    1116.00000
             2021-03-03       11.00000    1116.00000
             2021-06-19       11.00000    1116.00000
             2021-09-21       11.00000    1116.00000

[3841480 rows x 2 columns]

In [16]:
filter_drop = filter_df.copy()
filter_drop_reset = filter_drop.reset_index()

In [17]:
new_df = pd.DataFrame()

In [18]:
filter_drop_reset.head()

,account,pay_date,day_payment,balance_sum,day_debt_bad_max,mrf_service_id,rf_service_id,sent_date,days_from_j,months_from_j,...,USDRUB_pct,brent_pct,EURRUB_pct,invst,key_rate,target_before_sent,target_3m,target_6m,target_12m,target_24m
0,2.100100e+11,2017-01-11,996.75,697.0,0,12.00000,1204.00000,2021-12-01,-1785.0,-60.0,...,-0.202594,0.603339,-0.205551,0.997395,10.00,0,0,0,0,0
1,2.100100e+11,2017-02-16,697.25,818.0,0,12.00000,1204.00000,2021-12-01,-1749.0,-58.0,...,-0.224560,0.545455,-0.240551,1.015000,10.00,0,0,0,0,0
2,2.100100e+11,2017-03-14,818.00,767.0,0,12.00000,1204.00000,2021-12-01,-1723.0,-57.0,...,-0.160870,0.334091,-0.213282,1.015000,10.00,0,0,0,0,0
3,2.100100e+11,2017-04-14,767.00,0.0,0,12.00000,1204.00000,2021-12-01,-1692.0,-56.0,...,-0.119604,0.074797,-0.163430,1.015000,9.75,0,0,0,0,0
4,2.100100e+11,2017-04-17,1000.00,0.0,0,12.00000,1204.00000,2021-12-01,-1689.0,-56.0,...,-0.119604,0.074797,-0.163430,1.015000,9.75,0,0,0,0,0


In [19]:
for idx in tqdm(filter_drop.index.get_level_values('account').unique()):
    index_to_drop = filter_drop_reset[filter_drop_reset['account'] == idx][-24:]
    
    #filter_drop.drop((idx, i.strftime('%Y-%m-%d')), axis=0, inplace=True)
    
    new_df = new_df.append(index_to_drop)

100%|█████████████████████████████████| 181443/181443 [2:42:45<00:00, 18.58it/s]


In [5]:
new_df.to_pickle('mid_term.pkl')

NameError: name 'new_df' is not defined

In [25]:
del data, data_reset, target, target_total, filter_df, filter_drop, filter_drop_reset,features

In [6]:
# start

In [5]:
new_df = pd.read_pickle('mid_term.pkl')

In [6]:
new_df_copy = new_df.copy()

In [7]:
new_df_copy = new_df_copy[pd.notnull(new_df_copy['balance_sum'])]

In [8]:
sort_idx_notna = new_df_copy.groupby('account').size().sort_values(ascending=False).loc[lambda x : x >=24].index

In [9]:
new_df_copy = new_df_copy[new_df_copy.account.isin(sort_idx_notna)]

In [10]:
target = new_df_copy[['account','target_24m']].drop_duplicates()

In [11]:
FEATURES = ['day_payment', 'balance_sum', 'day_debt_bad_max',
                      'ipi1', 'ipi2', 'ppi1', 'ppi2',
       'ppi3', 'cpi1', 'unemp1', 'gdp_nom', 'gdp_yoy', 'salary', 'sales',
       'income_nom', 'income_real_1', 'income_real_2', 'USDRUB1', 'EURRUB1',
       'brent', 'USDRUB_pct', 'brent_pct', 'EURRUB_pct', 'invst', 'key_rate']

In [12]:
X_train = new_df_copy[['account','day_payment', 'balance_sum', 'day_debt_bad_max',
                      'ipi1', 'ipi2', 'ppi1', 'ppi2',
       'ppi3', 'cpi1', 'unemp1', 'gdp_nom', 'gdp_yoy', 'salary', 'sales',
       'income_nom', 'income_real_1', 'income_real_2', 'USDRUB1', 'EURRUB1',
       'brent', 'USDRUB_pct', 'brent_pct', 'EURRUB_pct', 'invst', 'key_rate']]

In [13]:
sequences = []

for idx, group in tqdm(X_train.groupby('account')):
    
    seq_feat = group[FEATURES]
    label = target[target.account == idx].iloc[0].target_24m
    
    sequences.append((seq_feat, int(label)))

100%|█████████████████████████████████| 171724/171724 [01:07<00:00, 2528.23it/s]


In [14]:
train_seq, test_seq = train_test_split(sequences, test_size = 0.2)

## Dataset

In [22]:
from dataset_lstm import Lstm_Dataset

In [23]:
class SurfaceModule(LightningDataModule):
    
    def __init__(self, train_seq, test_seq, batch_size):
        super().__init__()
        self.train_seq = train_seq
        self.test_seq = test_seq
        self.batch_size = batch_size
        
    def setup(self, stage = None):
        self.train_dataset = Lstm_Dataset(self.train_seq)
        self.test_dataset = Lstm_Dataset(self.test_seq)
    
    def train_dataloader(self):
        
        return( DataLoader(
        self.train_dataset, 
        batch_size = self.batch_size,
        shuffle = True, num_workers = 2))
    
    def val_dataloader(self):
        
        return( DataLoader(
        self.test_dataset, 
        batch_size = self.batch_size,
        shuffle = False, num_workers = 2))
    
    def test_dataloader(self):
        
        return( DataLoader(
        self.test_dataset, 
        batch_size = self.batch_size,
        shuffle = False, num_workers = 2))

In [24]:
EPOCHS = 20
BATCH_SIZE = 256

data_module = SurfaceModule(train_seq, test_seq, BATCH_SIZE)

## Model

In [25]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

In [26]:
class LSTM_Model(nn.Module):
    
    def __init__(self, n_features, n_classes, n_hidden = 256, n_layers = 3):
        super().__init__()
        
        self.lstm = nn.LSTM(input_size = n_features, 
                           hidden_size = n_hidden,
                           num_layers = n_layers, 
                           batch_first = True, 
                           dropout = 0.75)
        self.linear = nn.Linear(n_hidden, n_classes)
        
    def forward(self, x):
        
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        
        return self.linear(hidden[-1])
    

In [27]:
class LSTM_PL(LightningModule):
    def __init__(self, n_features, n_classes):
        super().__init__()

            
        self.model = LSTM_Model(n_features, n_classes)
        self.criterion = nn.CrossEntropyLoss()


    def forward(self, x):
        
        output = self.model(x['sequence'])

        return output

    def training_step(self, batch, *args):
        
        sequences = batch['sequence']
        labels = batch['label']
        
        output = self.forward(batch)
        loss = self.criterion(output, batch['label'].to(device))
        predictions = torch.argmax(output, dim = 1)
        step_acc = accuracy_score(predictions, batch['label'])
        
        self.log("training_loss", loss.item(), prog_bar = True, logger = True)
        self.log("training_accuracy", step_acc.item(), prog_bar = True, logger = True)
        return {"loss": loss, "accuracy" : step_acc}
    

    @torch.no_grad()
    def validation_step(self, batch, *args):
        
        sequences = batch['sequence']
        labels = batch['label']
        
        output = self.forward(batch)
        loss = self.criterion(output, batch['label'].to(device))
        predictions = torch.argmax(output, dim = 1)
        step_acc = accuracy_score(predictions, batch['label'])
        
        self.log("val_loss", loss.item(), prog_bar = True, logger = True)
        self.log("val_accuracy", step_acc.item(), prog_bar = True, logger = True)
        return {"loss": loss, "accuracy" : step_acc}
    
    @torch.no_grad()
    def test_step(self, batch, *args):
        
        sequences = batch['sequence']
        labels = batch['label']
        
        output = self.forward(batch)
        loss = self.criterion(output, batch['label'].to(device))
        predictions = torch.argmax(output, dim = 1)
        step_acc = accuracy_score(predictions, batch['label'])
        
        self.log("test_loss", loss.item(), prog_bar = True, logger = True)
        self.log("test_accuracy", step_acc.item(), prog_bar = True, logger = True)
        return {"loss": loss, "accuracy" : step_acc}
    
    def configure_optimizers(self):
        
        
        optimizer = torch.optim.AdamW(self.parameters(), lr=3e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size = 10, gamma = 0.1)
        return {"optimizer": optimizer, 
               "lr_scheduler": {
                   "scheduler": scheduler,
                   "monitor": "training_loss"
               }}

In [28]:
model = LSTM_PL(
n_features = len(FEATURES),
n_classes = 2).to(device)

In [29]:
trainer = Trainer(max_epochs = EPOCHS, gradient_clip_val=0.1)
trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | LSTM_Model       | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.372     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
